In [ ]:

import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
df=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")

In [ ]:
df.head()

In [ ]:
df=df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [ ]:
df.head()

In [ ]:
df['v2'][0]

In [ ]:
df['v2'][2]

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
sns.countplot('v1',data=df)
plt.title("No. of ham vs spam emails")

In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
import re
import string

In [ ]:
df['v2'] = df['v2'].apply(clean)
df.head()

In [ ]:
from nltk.corpus import stopwords


In [ ]:
stopword=set(stopwords.words('english'))


In [ ]:
stopword

In [ ]:
def remove_stopwords(text):
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    return text

In [ ]:
df['v2'] = df['v2'].apply(remove_stopwords)
df.head()

In [ ]:
import nltk

In [ ]:
stemmer = nltk.SnowballStemmer("english")

In [ ]:
stemmer

In [ ]:
def stemming(text):
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [ ]:
df['v2'] = df['v2'].apply(stemming)
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['v1']=le.fit_transform(df['v1'])
df.head()

In [ ]:
le.classes_

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
df[df['v1']==0]['v2']

In [ ]:
x=df['v2']
y=df['v1']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english', ngram_range=(1,3))
x_train_vectorizer=count.fit_transform(x_train)

In [ ]:
x_test_vectorizer=count.transform(x_test)

In [ ]:
x_train_vectorizer.toarray()

In [ ]:
count.vocabulary_

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
tfidf = TfidfTransformer()

x_train_tfidf = tfidf.fit_transform(x_train_vectorizer)

x_train_tfidf.toarray()

In [ ]:
x_test_tfidf = tfidf.transform(x_test_vectorizer)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [ ]:
model_vectorizer= MultinomialNB().fit(x_train_vectorizer, y_train)
prediction_vectorizer=model_vectorizer.predict(x_test_vectorizer)
print(confusion_matrix(y_test,prediction_vectorizer))
print (classification_report(y_test, prediction_vectorizer))


In [ ]:
model_tfidf= MultinomialNB().fit(x_train_tfidf, y_train)
prediction_tfidf=model_tfidf.predict(x_test_tfidf)
print (classification_report(y_test, prediction_tfidf))
print(confusion_matrix(y_test,prediction_tfidf))

In [ ]:
import xgboost as xgb


In [ ]:
xgb_model=xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc' )

In [ ]:
xgb_model_vectorizer = xgb_model.fit(x_train_vectorizer, y_train)
xgb_predictions_vectorizer=xgb_model_vectorizer.predict(x_test_vectorizer)
print(confusion_matrix(y_test,xgb_predictions_vectorizer))
print (classification_report(y_test, xgb_predictions_vectorizer))

In [ ]:
xgb_model = xgb_model.fit(x_train_tfidf, y_train)
xgb_predictions=xgb_model.predict(x_test_tfidf)
print(confusion_matrix(y_test,xgb_predictions))
print (classification_report(y_test, xgb_predictions))

In [ ]:

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,SpatialDropout1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [ ]:
max_words = 1000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix

In [ ]:
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

In [ ]:
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=10)])

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
lstm_prediction=model.predict(test_sequences_matrix)

In [ ]:
lstm_prediction

In [ ]:
for prediction in lstm_prediction:
    print(prediction[0])

In [ ]:
res=[]
for prediction in lstm_prediction:
    if prediction[0]<0.5:
        res.append(0)
    else:
        res.append(1)

In [ ]:
print(confusion_matrix(y_test,res))

In [ ]:
max_words = 50000
max_len = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
from keras.models import Sequential


In [ ]:
# inputs = Input(name='inputs',shape=[max_len])
# layer = Embedding(max_words,50,input_length=max_len)(inputs)
# layer = LSTM(64)(layer)
# layer = Dense(256,name='FC1')(layer)
# layer = Activation('relu')(layer)
# layer = Dropout(0.5)(layer)
# layer = Dense(1,name='out_layer')(layer)
# layer = Activation('sigmoid')(layer)
# model = Model(inputs=inputs,outputs=layer)
model2 = Sequential()
model2.add(Embedding(max_words, 100, input_length=max_len))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()
model2.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
history2=model2.fit(sequences_matrix,y_train,batch_size=256,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=4)])

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model2.evaluate(test_sequences_matrix,y_test)

In [ ]:
lstm2_prediction=model2.predict(test_sequences_matrix)

In [ ]:
res2=[]
for prediction in lstm2_prediction:
    if prediction[0]<0.5:
        res2.append(0)
    else:
        res2.append(1)

In [ ]:
print(confusion_matrix(y_test,res2))

In [ ]:
model2.save("Spam_Classifier_model.h5")

In [ ]:
import keras

In [ ]:
load_model=keras.models.load_model("./Spam_Classifier_model.h5")

In [ ]:
test = 'Get free ringtone like never before get free '
def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
test=[clean_text(test)]
print(test)
seq = tokenizer.texts_to_sequences(test)
padded = sequence.pad_sequences(seq, maxlen=1000)
pred = load_model.predict(padded)
print("pred", pred)
if pred<0.5:
    print("ham")
else:
    print("spam")